The idea is to end with deleting all the rows from \[FinancialTransaction\] that have been created on or after '2024-03-13 00:00:00.000'

Therefore, these are all the rows from \[FinancialTransaction\] that should be deleted:

In [5]:
SELECT
    COUNT(*) AS 'Count FinancialTransaction'
FROM
    [FinancialTransaction] ft
WHERE
    ft.[CreatedDateTime] >= '2024-03-13 00:00:00.000'
    AND
    ft.[SourceTypeValueId] NOT IN (513, 577)
;

(1 row affected)

Total execution time: 00:00:00.095

Count FinancialTransaction
1226


This second SQL query shows all the rows from \[FinancialTransactionDetail\] that should be deleted:

<u>Note</u>:

I expected this COUNT to match the COUNT from \[FinancialTransaction\], but it doesn't.

There are fewer \[FinancialTransactionDetail\] rows than there are \[FinancialTransaction\] rows. Therefore, there are some FinancialTransactions without a matching Detail.

In [6]:
WITH FilteredTransactions AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000' AND [SourceTypeValueId] NOT IN (513, 577))

SELECT
    COUNT(*) AS 'Count FinancialTransactionDetail'
FROM
    [FinancialTransactionDetail] ftd
WHERE
    ftd.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions)
;

(1 row affected)

Total execution time: 00:00:00.113

Count FinancialTransactionDetail
1213


This is the best way I know how to check whether there would be any rows in \[FinancialScheduledTransaction\] relating to \[FinancialTransaction\]

In [4]:
WITH FilteredTransactionCodes AS (SELECT [TransactionCode] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000')

SELECT
    COUNT(*) AS 'Count FinancialScheduledTransaction'
FROM
    [FinancialScheduledTransaction] fst
WHERE
    fst.[TransactionCode] IN (SELECT [TransactionCode] FROM FilteredTransactionCodes)
;

(1 row affected)

Total execution time: 00:00:00.077

Count FinancialScheduledTransaction
0


It's worth noting that, had I not included the ft.\[SourceTypeValueId\] among the filters in the initial selection of \[FinancialTransaction\], i would have deleted rows associated to "On-Site" and "Bank Checks".

  

There are 111 rows in \[FinancialTransactionImage\] associated to FinancialTransactions that occurred on or after 2024-03-13.

I want to keep these \[FinancialTransactionImage\] rows intact, which is why we are filtering by ft.\[SourceTypeValueId\]

  

This is how I double-checked and triple-checked my reasoning:

In [7]:
WITH FilteredTransactions0 AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000')

SELECT
    COUNT(*) AS 'Count FTI no filter'
FROM
    [FinancialTransactionImage] fti
WHERE
    fti.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions0)
;

WITH FilteredTransactions1a AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000' AND [SourceTypeValueId] NOT IN (513))

SELECT
    COUNT(*) AS 'Count FTI 1a filter'
FROM
    [FinancialTransactionImage] fti
WHERE
    fti.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions1a)
;

WITH FilteredTransactions1b AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000' AND [SourceTypeValueId] NOT IN (577))

SELECT
    COUNT(*) AS 'Count FTI 1b filter'
FROM
    [FinancialTransactionImage] fti
WHERE
    fti.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions1b)
;

WITH FilteredTransactions2 AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000' AND [SourceTypeValueId] NOT IN (513, 577))

SELECT
    COUNT(*) AS 'Count FTI 2 filters'
FROM
    [FinancialTransactionImage] fti
WHERE
    fti.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions2)
;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.277

Count FTI no filter
111


Count FTI 1a filter
70


Count FTI 1b filter
41


Count FTI 2 filters
0


There are also some other tables that are related to the \[FinancialTransaction\] table, but don't have any rows related to the duplication issue, therefore I will not be deleting any rows from these tables.

  

These tables are:

\[FinancialTransactionAlert\]

\[FinancialTransactionRefund\]

In [8]:
WITH FilteredTransactions AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000')

SELECT
    COUNT(*) AS 'Count FinancialTransactionAlert'
FROM
    [FinancialTransactionAlert] fta
WHERE
    fta.[TransactionId] IN (SELECT [Id] FROM FilteredTransactions)
;

WITH FilteredTransactions AS (SELECT [Id] FROM [FinancialTransaction] WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000')

SELECT
    COUNT(*) AS 'Count FinancialTransactionRefund'
FROM
    [FinancialTransactionRefund] ftr
WHERE
    ftr.[OriginalTransactionId] IN (SELECT [Id] FROM FilteredTransactions)
;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.103

Count FinancialTransactionAlert
0


Count FinancialTransactionRefund
0


**<mark><u>Summary</u></mark>**:

The only rows I need to delete are 1226 rows from \[FinancialTransaction\], and 1213 rows from \[FinancialTransactionDetail\]

And in order to do that, I need to delete the rows from \[FinancialTransactionDetail\] first, and then the rows from \[FinancialTransaction\] second.

These are the DELETE statements I used:

In [ ]:
WITH FilteredTransactions AS (
    SELECT [Id]
    FROM [FinancialTransaction]
    WHERE [CreatedDateTime] >= '2024-03-13 00:00:00.000'
    AND [SourceTypeValueId] NOT IN (513, 577)
)

DELETE FROM [FinancialTransactionDetail]
WHERE [TransactionId] IN (
    SELECT [Id]
    FROM FilteredTransactions
);


In [ ]:
DELETE FROM [FinancialTransaction]
WHERE
    [CreatedDateTime] >= '2024-03-13 00:00:00.000'
    AND
    [SourceTypeValueId] NOT IN (513, 577)
;

These queries successfully deleted the 1213 rows from \[FinancialTransactionDetail\] and 1226 rows from \[FinancialTransaction\]